
# Bases des télécommunications 1

Date : 10/11/2023

Étudiants : Meissner, Volta


## __Introduction__

Le but de ce laboratoire est de mettre en pratique les concepts de compression de données vus en classe de Bases des
Télécommunications. On travaillera sur la compression d’un fichier texte

- Le rendu du **jupyter notebook** doit se faire sur __cyberlearn__ sous le format de nom de fichier "telecom_labo2_\<nom1\>_\<nom2\>.ipynb" 
- Le fichier texte utilisé ici doit être encodé en UTF-8 (sans BOM), vous pouvez le vérifier avec Notepad++ par exemple.
- Exemple de code python 3 pour ouvrir en lecture un fichier texte en UTF-8 : __*open(INPUT, "r", encoding="UTF-8")*__


# III. Shannon-Fano

On veut construire un code de Shannon-Fano pour compresser notre fichier texte. On part du
principe que l'arbre de codage généré n’a pas besoin d’être transmis dans le fichier compressé.

a) Comme le codage de Huffman, réalisez ci-dessous un programme qui lit un fichier texte, puis générez un codage de
Shannon-Fano pour les caractères de ce fichier. Votre programme doit afficher le code
binaire choisi pour chaque caractère.


Exemple :

<table>
<thead>
    <tr>
        <th>char</th>
        <th>hex</th>
        <th>binary</th>
    </tr>
</thead>
<tr>
    <td>a</td>
    <td>0x61</td>
    <td>11</td>
</tr>
<tr>
    <td>b</td>
    <td>0x62</td>
    <td>10</td>
</tr>
<tr>
    <td>@</td>
    <td>0x40</td>
    <td>00</td>
</tr>
<tr>
    <td>c</td>
    <td>0x63</td>
    <td>011</td>
</tr>
<tr>
    <td>A</td>
    <td>0x41</td>
    <td>010</td>
</tr>
</table>

In [284]:
# exercice a.
inputList = []
with open("input.txt", "r", encoding="UTF-8") as file:
    contents = file.read()
    inputList = list(contents) # Convert the file into an array of char


sortedByFrequence = {}

for c in inputList:
    sortedByFrequence[c] = sortedByFrequence.get(c, 0) + 1

 # sortedByFrequence.sortedByItsValue()???? <--- A dictionnary CAN'T be sort by his values

sortedByFrequence = sorted(sortedByFrequence.items(), key=lambda x: x[1], reverse=True) # So using a lambda (that will create tuples and order by the second value in the tuple (which is the value))
# was {'L': 3, 't': 34, 'g': 2, ...} <- Dictionnary
# now [(' ', 61), ('e', 49), ('t', 34), ...] <- List of Tuple

with open("out2.txt", "w") as file:
    for (char, frequency) in sortedByFrequence:
        for _ in range(0, frequency):
            file.write(char)



sumFrequencies = sum(item[1] for item in sortedByFrequence)

print(f"char | ascii | frequency")
for (char, frequency) in sortedByFrequence:
    print(f"{char} | {hex(ord(char))} | {round(frequency / sumFrequencies, 3)}")



shannonReady = [(char, frequency / sumFrequencies) for char, frequency in sortedByFrequence]


char | ascii | frequency
  | 0x20 | 0.151
e | 0x65 | 0.122
t | 0x74 | 0.084
u | 0x75 | 0.062
s | 0x73 | 0.06
a | 0x61 | 0.057
n | 0x6e | 0.057
r | 0x72 | 0.05
i | 0x69 | 0.047
o | 0x6f | 0.042
l | 0x6c | 0.037
d | 0x64 | 0.03
m | 0x6d | 0.03
p | 0x70 | 0.022
c | 0x63 | 0.017
. | 0x2e | 0.017
' | 0x27 | 0.012
x | 0x78 | 0.012
é | 0xe9 | 0.012
v | 0x76 | 0.01
L | 0x4c | 0.007
I | 0x49 | 0.007
q | 0x71 | 0.007
D | 0x44 | 0.007
b | 0x62 | 0.007

 | 0xa | 0.007
g | 0x67 | 0.005
C | 0x43 | 0.005
è | 0xe8 | 0.002
f | 0x66 | 0.002
ç | 0xe7 | 0.002
à | 0xe0 | 0.002
, | 0x2c | 0.002


In [285]:
# exercice a.
class Node:
   def __init__(self, data):
      self.left = None
      self.right = None
      self.data = data


def ShowTreeOfTuple(binaryCode, node: Node, array):
   if (node.data[0] == ''):
      if node.left:
         ShowTreeOfTuple(binaryCode + "0", node.left, array)
      if node.right:
         ShowTreeOfTuple(binaryCode + "1", node.right, array)
   else:
      array.append((node.data[0], node.data[1], hex(ord(node.data[0])), binaryCode))
      return


def shannonFano(array: list[tuple], node: Node):
   sum = 0
   tmp = []    
   i = 0
   midValue = node.data[1] / 2;

   for i in range(len(array) - 1):
      sum += array[i][1]
      tmp.append(array[i])

      if abs(midValue - sum) < abs(midValue - (sum + array[i + 1][1])):
         if (len(tmp) == 1):
             node.left = Node(array[i])
         else:
             nodeGroup1 = Node(('', sum))
             node.left = nodeGroup1

             shannonFano(tmp, nodeGroup1)
         break;

   tmp = []
   sum = 0

   for j in range(i+1, len(array)):
      sum += array[j][1]
      tmp.append(array[j])

   if (len(tmp) == 1):
      node.right = Node(array[j])
   else:
      nodeGroup2 = Node(('', sum))
      node.right = nodeGroup2

      shannonFano(tmp, nodeGroup2)
   
   return


In [286]:
root = Node(('', 1))

shannonFano(shannonReady, root)

shannonArray = []
ShowTreeOfTuple("", root, shannonArray)

lm = 0

for s in shannonArray:
   lm += (len(s[3]) * s[1])
   print(f"{s[0]} | {s[1]} | {s[2]} | {s[2]} | {len(s[3])}")

  | 0.1513647642679901 | 0x20 | 0x20 | 3
e | 0.12158808933002481 | 0x65 | 0x65 | 3
t | 0.08436724565756824 | 0x74 | 0x74 | 3
u | 0.062034739454094295 | 0x75 | 0x75 | 4
s | 0.05955334987593052 | 0x73 | 0x73 | 4
a | 0.05707196029776675 | 0x61 | 0x61 | 4
n | 0.05707196029776675 | 0x6e | 0x6e | 4
r | 0.04962779156327544 | 0x72 | 0x72 | 4
i | 0.04714640198511166 | 0x69 | 0x69 | 5
o | 0.04218362282878412 | 0x6f | 0x6f | 5
l | 0.03722084367245657 | 0x6c | 0x6c | 5
d | 0.02977667493796526 | 0x64 | 0x64 | 5
m | 0.02977667493796526 | 0x6d | 0x6d | 5
p | 0.022332506203473945 | 0x70 | 0x70 | 6
c | 0.017369727047146403 | 0x63 | 0x63 | 6
. | 0.017369727047146403 | 0x2e | 0x2e | 6
' | 0.01240694789081886 | 0x27 | 0x27 | 6
x | 0.01240694789081886 | 0x78 | 0x78 | 6
é | 0.01240694789081886 | 0xe9 | 0xe9 | 7
v | 0.009925558312655087 | 0x76 | 0x76 | 7
L | 0.007444168734491315 | 0x4c | 0x4c | 7
I | 0.007444168734491315 | 0x49 | 0x49 | 7
q | 0.007444168734491315 | 0x71 | 0x71 | 7
D | 0.007444168734491315 | 

b) Afficher le taux moyen de bits par caractère de votre code, dans la console.

In [287]:
# exercice b.
import math


entropyH = 0
for (char, frequency) in sortedByFrequence:
    entropyH += frequency / sumFrequencies * math.log2(frequency / sumFrequencies)

entropyH = entropyH * -1 # Because the log makes the entropy negative



print(f"Taux moyen est lm = {lm}, H = {entropyH}")
print(f"R = {lm} - {entropyH} = {lm - entropyH} | H - lm")

Taux moyen est lm = 4.300248138957818, H = 4.266154878255668
R = 4.300248138957818 - 4.266154878255668 = 0.03409326070215002 | H - lm


 c) Que pouvez-vous conclure du résultat obtenu à l'exercice b) et de l’entropie du point (I.c), en une phrase ?


On est sous-optimal, car on a une redondance résiduel > 0, donc encore possible d'optimiser avec un code peut être optimal ?

d) Compresser le fichier texte d’input dans un nouveau fichier binaire à l'aide de votre arbre de codage. Si la longueur du message codé n’est pas un multiple de 8, rajouter des bits 0 à la fin. Le
fichier compressé doit avoir une taille sur disque significativement plus petite que
celui de base.

In [288]:
# exercice c.
outputList = []
summ = 0

for c in inputList:
    for shan in shannonArray:
        if shan[0] == c:
            outputList.append(shan[3])
            summ += len(shan[3])
            break

numberOfAddedZero = 0 if (summ % 8) == 0 else 8 - (summ % 8) 

outputString = ""
for s in outputList:
    outputString += s

outputString += "0" * numberOfAddedZero

byteLength = (len(outputString) + 7) // 8


with open("shannon.bin", "wb") as file:
    file.write(int(outputString, 2).to_bytes(byteLength, byteorder="big"))


e) Que manquerait-il a ce fichier binaire pour pouvoir être décodé par n’importe qui ?
(Bonus)

Il manque:
- Savoir comment on a diviser le tableau/Ou est la frontière (quel arrondi utilisé, ensuite si égal, dessus ou dessous)
- Quel côté on met le 0 et 1
- Pour les fréquences similaire, l'ordre utilisé (alphabétique, non-alphabétique, ...)


Et que l'autre personne décode en utilisant Shannon-Fano **ET** en utilisant ces mêmes règles

# IV. Bonus

⚠️ Difficile ⚠️ Pas obligatoire

Implémentez par vos moyens la compression de Lempel-Ziv-Welch ([Wikipedia](https://en.wikipedia.org/wiki/Lempel%E2%80%93Ziv%E2%80%93Welch)) et réalisez une étude comparative de cette méthode avec celles vues en cours.